In [141]:
# import txt data Cruz's speech
import io
with io.open('speech.txt', 'r', encoding="utf-8") as myfile:
    txt=myfile.read().replace('\n', '')

In [19]:
import numpy as np
import pandas as pd
import os
import re 
import string
from operator import itemgetter
import itertools
from collections import Counter
from pprint import pprint

# import NLTK package
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

# import SpaCy package
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 10000000000
from spacy import displacy

# sentiment analysis package
import pattern
from pattern.en import sentiment, positive

### 1 NER-tag your texts in NLTK and SpaCy and provide totals for each NER tag in both systems. (Please also include the total number of tokens for your corpus in NLTK and SpaCy.)

In [32]:
# NLTK
nltk_tok = nltk.word_tokenize(txt)
nltk_tok = [word for word in nltk_tok if word.isalpha()]

nltk_pos = nltk.pos_tag(nltk_tok)
nltk_posdf = pd.DataFrame(nltk_pos)

In [33]:
# NLTK number of tokens
print("The number of tokens using NLTK is ", len(nltk_tok))

The number of tokens using NLTK is  201964


In [36]:
nltk_chunks = nltk.ne_chunk(nltk_pos)
nltk_label = []
for i in nltk_chunks:
    if hasattr(i, 'label'):
        nltk_label.append(i.label())

In [38]:
# Totals for each NER tag in NLTK system
pprint(Counter(nltk_label))

Counter({'GPE': 3682,
         'PERSON': 3452,
         'ORGANIZATION': 2667,
         'FACILITY': 112,
         'LOCATION': 14,
         'GSP': 14})


In [146]:
# Spacy
spacy_label = [i.label_ for i in text_spacy.ents]

In [147]:
# Spacy number of tokens
print("The number of tokens using NLTK is ", len(text_spacy))

The number of tokens using NLTK is  233136


In [148]:
# Totals for each NER tag in NLTK system
pprint(Counter(spacy_label))

Counter({'GPE': 3486,
         'PERSON': 3066,
         'NORP': 2114,
         'DATE': 2030,
         'ORG': 1904,
         'CARDINAL': 1764,
         'TIME': 728,
         'LAW': 539,
         'ORDINAL': 427,
         'MONEY': 259,
         'FAC': 84,
         'LOC': 84,
         'WORK_OF_ART': 84,
         'EVENT': 56,
         'LANGUAGE': 49,
         'PRODUCT': 49,
         'PERCENT': 42})


### 2. Provide a few examples for each type of tag (if possible).

In [66]:
# NLTK
dfnltk = pd.DataFrame({'Word' :[],'Label':[]})
for i in nltk_chunks:
    if hasattr(i, 'label'):
        dfnltk = dfnltk.append({'Word': i[0][0], 'Label':i.label()}, ignore_index=True)

In [74]:
# NLTK example
dfnltk.drop_duplicates(['Word', 'Label']).groupby('Label').head(3).sort_values(by=['Label'])

,Label,Word
4,FACILITY,Oval
465,FACILITY,White
783,FACILITY,Fifth
2,GPE,America
6,GPE,Israel
7,GPE,Jerusalem
718,GSP,Syria
88,LOCATION,North
5,ORGANIZATION,American
9,ORGANIZATION,ISIS


In [149]:
# Spacy
dfspacy = pd.DataFrame({'Word' :[],'Label':[]})
for i in text_spacy.ents:
    dfspacy = dfspacy.append({'Word': i.text, 'Label':i.label_}, ignore_index=True)

In [150]:
# Spacy example
dfspacy.drop_duplicates(['Word', 'Label']).groupby('Label').head(3).sort_values(by=['Label'])

,Label,Word
12,CARDINAL,three
13,CARDINAL,four
11,CARDINAL,two
62,DATE,more than two centuries
65,DATE,the ages
72,DATE,the 1950s
1582,EVENT,Super Tuesday
556,EVENT,World War I.
1805,EVENT,the Second Amendment
525,FAC,the Emanuel AME Church


### 3 Find the 2 longest sentences in your text (either by number of tokens or raw length) and provide visual "trees" for them in either NLTK and SpaCy (extra credit if you can provide both).

In [152]:
# Using Spacy
sent_spacy = list(set([i for i in text_spacy.sents]))
dic_s={}
for i in sent_spacy:
    if len(i) not in dic_s:
        dic_s[len(i)] = [i]
    else:
        dic_s[len(i)].append(i)

In [182]:
k_s = list(dic_s.keys())
heapq.nlargest(3, k_s)

[92, 84, 78]

In [159]:
# Two longest sentence for Spacy
print(dic_s[92][0],'\n\n',dic_s[84][0])

If we commit to do more than hope, if we aspire to govern by a higher standard, a standard that transcends political party and race and gender and income, and to everyone that is tired of a president that divides us daily on racial lines, on ethnic lines, on religious lines, on gender lines, on socioeconomic lines, we will have a president that once again seeks to bring us together and unite us on our shared values of who we are as Americans. 

 And I want to speak for a moment to the soldiers and sailors and airmen and marines, I want to speak to the moms and dads who have a child serving overseas, I want to speak to the husbands and wives who wake up at night wondering if their loved one is still alive, I want to speak to the little kids who have been waiting months upon months to get a hug from daddy and a kiss from mommy.


In [160]:
s1 = nlp(str(dic_s[92][0]))
s2 = nlp(str(dic_s[84][0]))

In [165]:
# Tree for s1
displacy.render(s1,style = 'dep',jupyter = True, options = {"distance":65})

In [166]:
# Tree for s2
displacy.render(s2,style = 'dep',jupyter = True, options = {"distance":65})

### 4 Provide examples of different NER tags from your text for the “same” word/token as tagged in NLTK and SpaCy. (Extra credit for producing statistics for all differences.)

In [176]:
pd_merge = dfnltk.merge(dfspacy, on = 'Word', suffixes=['_NLTK','_Spacy']).drop_duplicates()

In [178]:
pd_different = pd_merge.query('Label_NLTK != Label_Spacy')

In [180]:
# Here are some examples
pd_different[:20]

,Label_NLTK,Word,Label_Spacy
98,ORGANIZATION,Reagan,PERSON
32599,ORGANIZATION,America,GPE
86653,PERSON,America,GPE
574525,ORGANIZATION,American,NORP
575407,GPE,American,NORP
785960,ORGANIZATION,Islamic,NORP
807569,ORGANIZATION,IRS,ORG
812420,GPE,Cuban,NORP
812616,PERSON,Caroline,ORG
834180,PERSON,Catherine,GPE


### 5. In a short paragraph, (~5 sentences), provide your thoughts on the NER tags in both NLTK and SpaCy and the strengths and weaknesses of each NER tagging system. Please also comment on the strengths and weaknesses, and the potential utility of NER as a whole.

Spacy has more tags than NLTK and based on the results of my text, Spacy's tags are more accurate. For example, "America" is definitely a country's name as my data is the speech of an American politician and Spacy correctly group it into "GPE". But NLTK classifies this word as a 'PERSON'. I am also amazed by the large vocabulary pool in Spacy. For example, Spacy could recognize both "Reagan" and "God" as person. However, both of them make mistakes. I prefer to use Spacy for the initial data process. NER maybe contribute to topic modelings and could give us a general sense of politicians' word preference.

### 6 Please install and use either TextBlob or parallel, and provide the overall sentiment scores (polarity & subjectivity) plus the detailed assessments for at least 3 sentences (or sections/passages) of your text, then in a sentence or 2, comment on the score and whether it seems to match your feelings about the sentiment of the source.

In [199]:
# Sentences
sent_1 = str(dic_s[92][0])
sent_2 = str(dic_s[84][0])
sent_3 = str(dic_s[78][0])

In [205]:
# analyze sent_1
print("The first sentece is:\n")
print(sent_1)
print('\n')
print("The overal sentiment scores is", sentiment(sent_1))
print("\n")
print("If the sentence is positive",positive(sent_1, threshold=0.1))
print("\n")
print("The detailed assesments are:")
print(sentiment(sent_1).assessments)

The first sentece is:

If we commit to do more than hope, if we aspire to govern by a higher standard, a standard that transcends political party and race and gender and income, and to everyone that is tired of a president that divides us daily on racial lines, on ethnic lines, on religious lines, on gender lines, on socioeconomic lines, we will have a president that once again seeks to bring us together and unite us on our shared values of who we are as Americans.


The overal sentiment scores is (0.04375, 0.25625)


If the sentence is positive False


The detailed assesments are:
[(['more'], 0.5, 0.5, None), (['higher'], 0.25, 0.5, None), (['standard'], 0.0, 0.0, None), (['standard'], 0.0, 0.0, None), (['political'], 0.0, 0.1, None), (['tired'], -0.4, 0.7, None), (['daily'], 0.0, 0.0, None), (['religious'], 0.0, 0.25, None)]


In [204]:
# analyze sent_2
print("The second sentece is:\n")
print(sent_2)
print('\n')
print("The overal sentiment scores is", sentiment(sent_2))
print("\n")
print("If the sentence is positive",positive(sent_2, threshold=0.1))
print("\n")
print("The detailed assesments are:")
print(sentiment(sent_2).assessments)

The second sentece is:

And I want to speak for a moment to the soldiers and sailors and airmen and marines, I want to speak to the moms and dads who have a child serving overseas, I want to speak to the husbands and wives who wake up at night wondering if their loved one is still alive, I want to speak to the little kids who have been waiting months upon months to get a hug from daddy and a kiss from mommy.


The overal sentiment scores is (0.20416666666666664, 0.5666666666666668)


If the sentence is positive True


The detailed assesments are:
[(['loved'], 0.7, 0.8, None), (['alive'], 0.1, 0.4, None), (['little'], -0.1875, 0.5, None)]


In [206]:
# analyze sent_3
print("The third sentece is:\n")
print(sent_3)
print('\n')
print("The overal sentiment scores is", sentiment(sent_3))
print("\n")
print("If the sentence is positive",positive(sent_3, threshold=0.1))
print("\n")
print("The detailed assesments are:")
print(sentiment(sent_3).assessments)

The third sentece is:

This is only a role”, he is lying to us, then we as a Republican Party would have failed profoundly for this country because the men and women of Indiana know if we get people a clear choice between freedom and tyranny, between prosperity and poverty, between vulnerability and weakness and peace through strength that the American people will choose the Constitution and jobs and freedom and security over and over again.


The overal sentiment scores is (-0.06333333333333332, 0.5366666666666666)


If the sentence is positive False


The detailed assesments are:
[(['only'], 0.0, 1.0, None), (['failed'], -0.5, 0.3, None), (['profoundly'], 0.08333333333333333, 1.0, None), (['clear'], 0.10000000000000002, 0.3833333333333333, None), (['american'], 0.0, 0.0, None)]


Cruz is talking about what kind of president he could be in sentence 1, which could be understood as a criticism that the current president hasn't achived. The score matches my judgement.
The second sentence is hard to tell whether it's positive or negative and we could see that the positive and negative score does not differentiate much.
The third sentence is defitetely criticizing Democrats and the score perfectly reflects that. The sentiment analysis sucessfully captures words like "only", "failed". 